In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#importing required libraries
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
import cv2


In [ ]:
def resize_img(img_path, resize_path = None, x_dim = 224, y_dim = 224):
  img = imread(img_path)

  #greyscale to RGB
  if img.ndim == 2:
    img = np.atleast_3d(img)
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)

  resized_img = resize(img, (x_dim, y_dim))

  if(resize_path is not None):
    plt.imsave(resize_path, resized_img)


In [ ]:
#Create Folders Data

#General
PATH = '/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture'

#Si no está creada la carpeta, la crea
# ambas view
if not os.path.exists(PATH):
  os.makedirs(PATH)
  os.makedirs(PATH + '/Fracture')
  os.makedirs(PATH + '/Normal')

# A view
PATH_A = '/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture A View'

#Si no está creada la carpeta, la crea
if not os.path.exists(PATH_A):
  os.makedirs(PATH_A)
  os.makedirs(PATH_A + '/Fracture')
  os.makedirs(PATH_A + '/Normal')

# L view
PATH_L = '/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture L View'

#Si no está creada la carpeta, la crea
if not os.path.exists(PATH_L):
  os.makedirs(PATH_L)
  os.makedirs(PATH_L + '/Fracture')
  os.makedirs(PATH_L + '/Normal')




In [ ]:
#Redimensionamiento de las imagenes y despues copiarlas a la carpeta objetivo
FOLDER = '/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture/'

if os.path.exists(FOLDER):
  #Copiar ficheros de la carpeta conjunta a las separadas
  path, dirs, files = next(os.walk("/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Wrist Fracture Cropped"))
  for name in dirs:
    path2, dirs2, files2 = next(os.walk(os.path.join(path, name)))
    for files3 in files2:
      new_name_file = files3[7:-4] #extraer 'Cropped' .tif
      img_paths = os.path.join('/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Wrist Fracture Cropped', name, files3) 
      resize_paths = os.path.join('/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture/',name, 'Resized' + new_name_file + '.jpg')
        
      resize_img(img_path = img_paths, resize_path=resize_paths)

      if resize_paths.find('A View') > -1:
        file_dest_resize = resize_paths[:72] + ' A View/' + resize_paths[73:]

      if resize_paths.find('L View') > -1:
        file_dest_resize = resize_paths[:72] + ' L View/' + resize_paths[73:]

        
      shutil.copy(resize_paths, file_dest_resize)
      



In [ ]:
# Separacion de los datos en el training data y el test data
ROOTS_DIRS= ['/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture',
          '/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture A View',
          '/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture L View'
]

CLASES = ['Normal', 'Fracture']
for r in ROOTS_DIRS: 
  for i in CLASES:
    os.makedirs(r +'/train/' + i)
    os.makedirs(r +'/test/' + i)
    source =r + '/' + i
    allFileNames = os.listdir(source)
    np.random.shuffle(allFileNames)
    test_ratio = 0.20
    train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)* (1 - test_ratio))])

    train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
    test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]
    for name in train_FileNames:
      shutil.copy(name, r +'/train/' + i)

    for name in test_FileNames:
      shutil.copy(name, r +'/test/' + i)